In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.manifold import MDS
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
import os 

In [23]:
example_y = pd.read_parquet(r"C:\Users\ethor\Desktop\hackathon\OWL_Factory_Hackathon\submissions\test_dir_y\2023_03_16_target.pq")

In [24]:
example_y.head()

,ProzessData_ActData_AB1_Temperature_DR1_MassMixingStage
2023-03-16 01:00:00+00:00,41.5
2023-03-16 01:00:01+00:00,41.5
2023-03-16 01:00:02+00:00,41.5
2023-03-16 01:00:03+00:00,41.5
2023-03-16 01:00:04+00:00,41.5


In [25]:
from pathlib import Path
import numpy as np
import pickle
import pandas as pd 
import os 



def add_time_features(df):
    df['hour'] = df.index.hour
    df['minute'] = df.index.minute
    daytime_arr = []
    for idx in df.index:
        if df.loc[idx, "hour"] >= 6 and df.loc[idx, "hour"] <= 18:
            daytime_arr.append(1)
        else:
            daytime_arr.append(0)

    df['day_time'] = daytime_arr
    df['night_time'] = 1 - df['day_time']
    return df



dirpath = Path(r"C:\Users\ethor\Desktop\hackathon\OWL_Factory_Hackathon\submissions\test_dir_x")
PATH_TO_DAY_MODEL = Path(r"C:\Users\ethor\Desktop\hackathon\OWL_Factory_Hackathon\submissions\Models\linreg.pkl")
PATH_TO_NIGHT_MODEL = Path(r"C:\Users\ethor\Desktop\hackathon\OWL_Factory_Hackathon\submissions\Models\linreg_2.pkl")


# with open(PATH_TO_DAY_MODEL, 'rb') as file:
#     day_model = pickle.load(file)

# with open(PATH_TO_NIGHT_MODEL, 'rb') as file:
#     night_model = pickle.load(file)
    
    
first = True
for filename in os.listdir(dirpath):
    ret_dict = {}
    filepath = os.path.join(dirpath, filename)
    data = pd.read_parquet(filepath)

    data = add_time_features(data)
    data.index = pd.to_datetime(data.index)

    day_data = data[data.day_time == 1]
    night_data = data[data.day_time == 0]
    
    cumul_index = list(day_data.index) + list(night_data.index)


        

    day_preds= day_model.predict(day_data)
    night_preds = night_model.predict(night_data)
    ret_df = pd.DataFrame({"ret_data": list(day_preds) + list(night_preds)}, index=cumul_index)
    ret_df = ret_df.sort_index()




    ret_df.to_parquet(fr"C:\Users\ethor\Desktop\hackathon\OWL_Factory_Hackathon\submissions\test_dir_res\{filename}-pred.pq")

c:\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.3.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [26]:
first = True
dirr_path = Path(r"C:\Users\ethor\Desktop\hackathon\OWL_Factory_Hackathon\submissions\test_dir_y")
for filepath in os.listdir(dirr_path):
    full_path = os.path.join(dirr_path, filepath)
    temp_df = pd.read_parquet(full_path)
    print(len(temp_df))
    if first:
        whole_df_true = temp_df
        first = False
        continue
    whole_df_true = pd.concat([whole_df_true, temp_df], axis=0)

68401
68401
68401
68401
68401


In [27]:
first = True
dirr_path = Path(r"C:\Users\ethor\Desktop\hackathon\OWL_Factory_Hackathon\submissions\test_dir_res")
for filepath in os.listdir(dirr_path):
    full_path = os.path.join(dirr_path, filepath)
    temp_df = pd.read_parquet(full_path)
    print(len(temp_df))
    if first:
        whole_df_preds = temp_df
        first = False
        continue
    whole_df_preds = pd.concat([whole_df_preds, temp_df], axis=0)

68401
68401
68401
68401
68401


In [29]:
len(whole_df_true) == len(whole_df_preds)

True

In [31]:
whole_df_true.iloc[:, 0]

2023-03-16 01:00:00+00:00    41.5
2023-03-16 01:00:01+00:00    41.5
2023-03-16 01:00:02+00:00    41.5
2023-03-16 01:00:03+00:00    41.5
2023-03-16 01:00:04+00:00    41.5
                             ... 
2023-04-13 19:59:56+00:00    41.5
2023-04-13 19:59:57+00:00    41.5
2023-04-13 19:59:58+00:00    41.5
2023-04-13 19:59:59+00:00    41.5
2023-04-13 20:00:00+00:00    41.5
Name: ProzessData_ActData_AB1_Temperature_DR1_MassMixingStage, Length: 342005, dtype: float64

In [33]:
mae = mean_absolute_error(whole_df_true.iloc[:, 0], whole_df_preds.iloc[:, 0])
mae

0.10424721724899556

In [16]:
len(whole_df)

342005

In [15]:
len(ret_df)

68401

In [ ]:
day_preds

array([[34.29969937],
       [34.37228242],
       [34.38220385],
       ...,
       [41.49908496],
       [41.50107374],
       [41.49908496]])

In [ ]:
train_x = pd.read_csv(r"C:\Users\ethor\Desktop\hackathon\OWL_Factory_Hackathon\data\train_x.csv", index_col=0)

In [ ]:
train_y = pd.read_csv(r"C:\Users\ethor\Desktop\hackathon\OWL_Factory_Hackathon\data\train_y.csv", index_col=0)

In [ ]:
train_x[:1000].to_csv("train_x.csv")
train_y[:1000].to_csv("train_y.csv")

In [ ]:
train_y.index = pd.to_datetime(train_y.index)

In [ ]:
def add_time_features(df):
    df['hour'] = df.index.hour
    df['minute'] = df.index.minute
    daytime_arr = []
    for idx in df.index:
        if df.loc[idx, "hour"] >= 6 and df.loc[idx, "hour"] <= 18:
            daytime_arr.append(1)
        else:
            daytime_arr.append(0)

    df['day_time'] = daytime_arr
    df['night_time'] = 1 - df['day_time']
    return df

In [ ]:
train_x.index = pd.to_datetime(train_x.index)
train_y.index = pd.to_datetime(train_y.index)

In [ ]:
train_x = add_time_features(train_x)
train_y = add_time_features(train_y)

In [ ]:
train_x_day = train_x[train_x.day_time == 1]

In [ ]:
train_y_day = train_y[train_x.day_time == 1]

In [ ]:
train_x_day = train_x_day.drop(columns=["day_time", "night_time"])
train_y_day = train_y_day.drop(columns=["day_time", "night_time"])

In [ ]:
scaler = StandardScaler()
standardized_train = scaler.fit_transform(train_x_day)
scaled_df = pd.DataFrame(standardized_train, columns=train_x_day.columns)
scaled_df = scaled_df.drop(columns=["hour", "minute"])

In [ ]:
pca = PCA(n_components=len(scaled_df.columns))  # You can specify the number of components you want
X_pca = pca.fit_transform(scaled_df)


In [ ]:
model = SVR(kernel="rbf")

In [ ]:
model.fit(train_x_day[:500000], train_y_day.iloc[:,0][:500000])

In [ ]:
y_pred = model.predict(train_x_day[500000:510000])

In [ ]:
mean_absolute_error(train_y_day[500000:510000], y_pred)

0.09512403624058288